 # **Time Series Forecasting **

Time series analysis comprises methods for analyzing time series data in order to extract meaningful statistics and other characteristics of the data. Time series forecasting is the use of a model to predict future values based on previously observed values.

Time series are widely used for non-stationary data, like economic, weather, stock price, and retail sales in this post. 
**I am going to  predict Gold price with time series forecasting using RNN method**

 Time Series Forecasting using CNN-LSTM


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from keras.models import Sequential
import matplotlib.patches as mpatches
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error


import os


# Any results you write to the current directory are saved as output.

In [ ]:
data = pd.read_csv('/content/Gold_data.csv')

In [ ]:
data.columns

In [ ]:
data.head()

In [ ]:

data.tail()

In [ ]:
data['date'] = pd.to_datetime(data['date'])
data.set_index('date', inplace= True)
data =data.resample('D').mean()

In [ ]:
data = data[['price' ]]

In [ ]:
data.info()

In [ ]:
from scipy.interpolate import interp1d


In [ ]:
data['price'] = data['price'].interpolate()# we will fill the null row

In [ ]:
data.info()

In [ ]:
data.head()

In [ ]:
len(data)

In [ ]:
df = data.copy()

In [ ]:
plt.figure(figsize=(20,8))
plt.plot(data)
plt.title('Time Series')
plt.xlabel('Date')
plt.ylabel('temperature')
plt.show()

# Time Series Forecast using LSTM

Long Short Term Memory networks – usually just called “LSTMs” – are a special kind of RNN, capable of learning long-term dependencies. They were introduced by Hochreiter & Schmidhuber (1997), and were refined and popularized by many people in following work.1 They work tremendously well on a large variety of problems, and are now widely used.

LSTMs are explicitly designed to avoid the long-term dependency problem. Remembering information for long periods of time is practically their default behavior, not something they struggle to learn!

All recurrent neural networks have the form of a chain of repeating modules of neural network. In standard RNNs, this repeating module will have a very simple structure, such as a single tanh layer.


In [ ]:
data=data.values
data = data.astype('float32')

In [ ]:
scaler= MinMaxScaler(feature_range=(-1,1))
sc = scaler.fit_transform(data)

In [ ]:
sc 

In [ ]:
len(sc)                                                          

In [ ]:
len(df)

In [ ]:
timestep = 50

X= []
Y=[]


for i in range(len(sc)- (timestep)):
    X.append(sc[i:i+timestep])
    Y.append(sc[i+timestep])


X=np.asanyarray(X)
Y=np.asanyarray(Y)


k = 1950
Xtrain = X[:k,:,:]
Xtest = X[k:,:,:]    
Ytrain = Y[:k]    
Ytest= Y[k:]                                            

In [ ]:
print(Xtrain.shape)
print(Xtest.shape)

In [ ]:
print(Ytrain.shape)
print(Ytest.shape)

In [ ]:
Xtrain

In [ ]:
Ytrain

In [ ]:
print(Ytrain.shape)
print(Ytest.shape)

 # LSTM Model

In [ ]:
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential

In [ ]:
model = Sequential()

model.add(LSTM(32,activation = 'relu', input_shape= (50,1), return_sequences=True))
model.add(LSTM(32, activation='relu', return_sequences=True))
model.add(LSTM(32, activation='sigmoid', return_sequences=False))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [ ]:
model.summary()

In [ ]:
history = model.fit(Xtrain,Ytrain,epochs=300, verbose=1)

Epoch 1/300
60/60 [==============================] - 7s 59ms/step - loss: 0.1566
Epoch 2/300
60/60 [==============================] - 4s 60ms/step - loss: 0.0202
Epoch 3/300
60/60 [==============================] - 7s 112ms/step - loss: 0.0185
Epoch 4/300
60/60 [==============================] - 5s 83ms/step - loss: 0.0184
Epoch 5/300
60/60 [==============================] - 4s 60ms/step - loss: 0.0179
Epoch 6/300
60/60 [==============================] - 4s 59ms/step - loss: 0.0176
Epoch 7/300
60/60 [==============================] - 3s 58ms/step - loss: 0.0175
Epoch 8/300
60/60 [==============================] - 4s 59ms/step - loss: 0.0173
Epoch 9/300
60/60 [==============================] - 4s 59ms/step - loss: 0.0170
Epoch 10/300
60/60 [==============================] - 4s 59ms/step - loss: 0.0169
Epoch 11/300
60/60 [==============================] - 4s 59ms/step - loss: 0.0152
Epoch 12/300
60/60 [==============================] - 4s 60ms/step - loss: 0.0170
Epoch 13/300
60/60 [====

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(history.history['loss'])
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Model Accuracy")

In [ ]:
import pickle

In [ ]:
pickle.dump(model,open('lstm_model.pkl','wb'))

In [ ]:
model.save('lstm_mode_karan.hp5')

In [ ]:
preds= model.predict(Xtest)
preds = scaler.inverse_transform(preds)


Ytest=np.asanyarray(Ytest)  
Ytest=Ytest.reshape(-1,1) 
Ytest = scaler.inverse_transform(Ytest)


Ytrain=np.asanyarray(Ytrain)  
Ytrain=Ytrain.reshape(-1,1) 
Ytrain = scaler.inverse_transform(Ytrain)

mean_squared_error(Ytest,preds)

In [ ]:
pre= model.predict(Xtrain)
pre = scaler.inverse_transform(pre)

In [ ]:
plt.figure(figsize=(20,9))
plt.plot(Ytrain , 'blue', linewidth=5)
plt.plot(pre,'r' , linewidth=4)
plt.legend(('Train','Predicted'))
plt.title("price Prediction")
plt.show()

In [ ]:
plt.figure(figsize=(20,9))
plt.plot(Ytest , 'blue', linewidth=5)
plt.plot(preds,'r' , linewidth=4)
plt.legend(('Test','Predicted'))
plt.title("price Prediction")
plt.show()

In [ ]:
test = pd.DataFrame(Ytest,columns=['Actual'])
pred = pd.DataFrame(preds,columns=['Predict'])

In [ ]:
results = pd.concat([test,pred],axis=1)

In [ ]:
results.head(20)

In [ ]:
def insert_end(Xin,new_input):
    #print ('Before: \n', Xin , new_input )
    for i in range(timestep-1):
        Xin[:,i,:] = Xin[:,i+1,:]
    Xin[:,timestep-1,:] = new_input
    #print ('After :\n', Xin)
    return Xin


In [ ]:
df

In [ ]:

# this section for unknown future
#we are getting next 30 steps
from datetime import timedelta
future=30
forcast = []
Xin = Xtest[-1:,:,:]
time=[]
for i in range(future):
    out = model.predict(Xin, batch_size=1)    
    forcast.append(out[0,0]) 
    print(forcast)
    Xin = insert_end(Xin,out[0,0]) 
    time.append(pd.to_datetime(df.index[-1])+timedelta(days=i+1))


In [ ]:
forcasted_output=np.asanyarray(forcast)   
forcasted_output=forcasted_output.reshape(-1,1) 
forcasted_output = scaler.inverse_transform(forcasted_output)                  

In [ ]:
forcasted_output = pd.DataFrame(forcasted_output)
date = pd.DataFrame(time)
df_result = pd.concat([date,forcasted_output],axis=1)
df_result.columns = "Date","Forecasted"

In [ ]:
df.columns

In [ ]:
df_result

In [ ]:
plt.figure(figsize=(16,8))
plt.title('Gold price prediction')
plt.xlabel('Date', fontsize=18)
plt.ylabel('price' ,fontsize=18)
plt.plot(df['price'][2000:])
plt.plot(df_result.set_index('Date')[['Forecasted']],"r--")

In [ ]:
from sklearn.metrics import mean_squared_error
from statsmodels.tools.eval_measures import rmse
  

In [ ]:
# Calculate mean squared error
mean_squared_error(Ytest, pred)

In [ ]:
# Calculate root mean squared error
rmse(Ytest, pred)